<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/driverTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/My Drive/ConfidentMachineTranslation/flores
!pwd
!ls
import sys
sys.path.append('/content/drive/My Drive/ConfidentMachineTranslation/flores/src/')
!pip install fairseq sacrebleu sentencepiece fastBPE sacremoses xgboost

In [ ]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [2]:
##Methods for CLI with fairseq - WIP
import os
import glob
from tqdm.notebook import tqdm

class FairseqWrapper:

    def runFairseqGenerate(self, dataFolder, sourceLang, targetLang, fwModel, beam, lenpen, dataSet, removeBpe, saveFile):
        !fairseq-generate $dataFolder --source-lang $sourceLang --target-lang $targetLang \
        --path $fwModel --beam $beam --lenpen $lenpen --gen-subset $dataSet --remove-bpe=$removeBpe --sacrebleu > $saveFile

    def runFairseqScore(self, translations, groundTruth, saveFile, metric):
        !fairseq-score --sys $translations --ref $groundTruth --$metric > $saveFile
    
    def runFairseqPreprocessLM(self, srcdict, preprocCommand, bpeTranslations, dest):
        !fairseq-preprocess --only-source --srcdict $srcdict --$preprocCommand $bpeTranslations \
        --destdir $dest --workers 20
    
    def runFairseqPreprocessBinarize(self, srcLang, trgLang, srcdict, preprocCommand, bpeTranslations, destdir):
        !fairseq-preprocess --source-lang $srcLang --target-lang $trgLang --srcdict $srcdict\
        --$preprocCommand $bpeTranslations --destdir $destdir --joined-dictionary --workers 20

    def runFairseqEvalLM(self, preprocFile, lmModel, batchSize, maxTokens, dataSet, saveFile):
        !fairseq-eval-lm $preprocFile --path $lmModel --batch-size $batchSize --max-tokens $maxTokens \
        --output-word-probs --sample-break-mode eos --gen-subset $dataSet > $saveFile

    def deleteFolder(self, folder):
        print("Deleting contents in folder: " + str(folder))
        for f in tqdm(glob.glob(folder)):
            !rm $f


In [3]:
import translationUtils
from translation import Translation
import dataUtils
import classification
import thresholding
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import regression
import transformerUtils 

trainTranslations = dataUtils.createObjectsFromFile("valid")
testTranslations = dataUtils.createObjectsFromFile("test")

print("Train elements = " + str(len(trainTranslations)))
print("Test elements = " + str(len(testTranslations)))

Train elements = 2559
Test elements = 2835


In [4]:
params = {
    'hidden_dim': 128,
    'num_heads': 2,
    'feedforward_dim': 2048,
    'dim_k': 96,
    'dim_v': 96,
    'dim_q': 96,
    'max_length': 100,
    'batch_size': 100,
    'verbosity': 1,
    'lr': 5e-3,
    'eps': 1e-8,
    'epochs': 25
}
model = transformerUtils.getClassifierTransformer(trainTranslations, testTranslations, 15, params)

time = 0.6627747535705566, epoch 1, loss = 34.73843765258789, 39.76651692390442 seconds per 1 epochs
time = 1.3315394918123882, epoch 2, loss = 13.549247741699219, 40.12575650215149 seconds per 1 epochs
time = 1.998296860853831, epoch 3, loss = 13.515931129455566, 40.005321979522705 seconds per 1 epochs
time = 2.6677593986193338, epoch 4, loss = 13.178577423095703, 40.16760540008545 seconds per 1 epochs
time = 3.3306435028711956, epoch 5, loss = 13.123237609863281, 39.772926330566406 seconds per 1 epochs
time = 3.991691835721334, epoch 6, loss = 12.826220512390137, 39.66277456283569 seconds per 1 epochs
time = 4.647988299528758, epoch 7, loss = 12.177009582519531, 39.377650022506714 seconds per 1 epochs
time = 5.309111400445302, epoch 8, loss = 11.250484466552734, 39.6672739982605 seconds per 1 epochs
time = 5.962142479419708, epoch 9, loss = 10.085058212280273, 39.181755781173706 seconds per 1 epochs
time = 6.618878515561422, epoch 10, loss = 6.109194278717041, 39.40397906303406 secon